In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import numpy as np

In [2]:
#create an empty dataframe that will hold ESG scores
df = pd.DataFrame(index=['Environment', 'Emissions', 'Resource Use', 'Innovation', 'Social', 'Human Rights', 'Product Responsibiliy',
         'Workforce', 'Community', 'Governance', 'Management', 'Shareholders', 'CSR Strategy', 'ESG Score'])
df.index.rename('Metric', inplace=True)    
print(df)

Empty DataFrame
Columns: []
Index: [Environment, Emissions, Resource Use, Innovation, Social, Human Rights, Product Responsibiliy, Workforce, Community, Governance, Management, Shareholders, CSR Strategy, ESG Score]


In [3]:
#read in the spreadsheets with the company names
df_us = pd.read_csv('us_stocks.csv')
df_us

,ticker,name,is_etf,exchange
0,A,"Agilent Technologies, Inc.",NaN,NYSE
1,AA,Alcoa Corp,NaN,NYSE
2,AABA,Altaba Inc.,NaN,NASDAQ
3,AAC,"AAC Holdings, Inc.",NaN,NYSE
4,AAL,"American Airlines Group, Inc.",NaN,NASDAQ
...,...,...,...,...
6950,ZUO,"Zuora, Inc.",NaN,NYSE
6951,ZVO,Zovio Inc.,NaN,NASDAQ
6952,ZYME,Zymeworks Inc.,NaN,NYSE
6953,ZYNE,"Zynerba Pharmaceuticals, Inc.",NaN,NASDAQ


In [4]:
# remove the commas, dots and other things from the company names to comply with Refinitiv format
df_us['name'].replace(',','', regex=True, inplace=True)
df_us['name'].replace('\.','', regex=True, inplace=True)
df_us['name'].replace('Corporation','Corp', regex=True, inplace=True)
df_us['name'].replace('Company','Co', regex=True, inplace=True)
df_us['name'].replace('(The)','',regex=True, inplace=True)
df_us['name'].replace('rapeutics','therapeutics',regex=True, inplace=True)
df_us

,ticker,name,is_etf,exchange
0,A,Agilent Technologies Inc,NaN,NYSE
1,AA,Alcoa Corp,NaN,NYSE
2,AABA,Altaba Inc,NaN,NASDAQ
3,AAC,AAC Holdings Inc,NaN,NYSE
4,AAL,American Airlines Group Inc,NaN,NASDAQ
...,...,...,...,...
6950,ZUO,Zuora Inc,NaN,NYSE
6951,ZVO,Zovio Inc,NaN,NASDAQ
6952,ZYME,Zymeworks Inc,NaN,NYSE
6953,ZYNE,Zynerba Pharmaceuticals Inc,NaN,NASDAQ


In [5]:
#create a list out of the company names
names_list = df_us['name'].tolist()
names_list

['Agilent Technologies Inc',
 'Alcoa Corp',
 'Altaba Inc',
 'AAC Holdings Inc',
 'American Airlines Group Inc',
 'Altisource Asset Management Corp',
 'Atlantic American Corp',
 "Aaron's Co Inc",
 'Applied Optoelectronics Inc',
 'AAON Inc',
 'Advance Auto Parts Inc',
 'Apple Inc',
 'American Assets Trust Inc',
 'Almaden Minerals Ltd',
 'Atlas Air Worldwide Holdings',
 'Axon Enterprise Inc',
 'AllianceBernstein Holding LP',
 'ABB Ltd',
 'AbbVie Inc',
 'AmerisourceBergen Corp',
 'Ameris Bancorp',
 'Alcentra Capital Corp',
 'Abeona therapeutics Inc',
 'Abeona therapeutics Inc',
 'Ambev SA',
 'Asbury Automotive Group Inc',
 'Ability Inc',
 'ARCA biopharma Inc',
 'ABM Industries Incorporated',
 'ABIOMED Inc',
 'Arbor Realty Trust Inc',
 'Arbor Realty Trust',
 'Arbor Realty Trust',
 'Arbor Realty Trust',
 'Abbott Laboratories',
 'Allegiance Bancshares Inc',
 'Arbutus Biopharma Corp',
 'Associated Capital Group Inc',
 'Arcosa Inc',
 'ACADIA Pharmaceuticals Inc',
 'Acamar Partners Acquisition C

In [6]:
#launch automated Chrome browser and direct to website
chrome_driver = '/Users/tina/Desktop/Drivers/chromedriver'
driver = webdriver.Chrome(chrome_driver)

driver.get('https://www.refinitiv.com/en/sustainable-finance/esg-scores')
time.sleep(2)

<ipython-input-6-0514b05c7766>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


In [7]:
# iterate through all of the company names and get their ESG scores, then append them 
# as a new column to the empty df

testseries = []
myvals = []
counter = 0

for i in names_list:
    counter += 1
    element = driver.find_element(By.ID,"searchInput-1")
    element.send_keys(i)
    element.send_keys(Keys.RETURN)
    time.sleep(3)
    
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')
    
    myvals = soup.find_all("div", {"class": "table-align-right"})
    hidden = soup.find('div', hidden=True)
  
    time.sleep(1)
    
    if hidden == None:
        for x in myvals:
            testseries.append(x.string)
        myheaders = soup.find('h3', class_ = 'Heading Heading--m').find('strong').get_text()
        testseries.append(myheaders)
        df[i] = testseries
    else:
        pass
    
    time.sleep(1)
    
    testseries = []
    
    ddelement = driver.find_element(By.CLASS_NAME, "SearchInput-clearButton")
    ddelement.click()
    
    time.sleep(2)
    
    if counter % 100 == 0:
        driver.quit()
        time.sleep(15)
        chrome_driver = '/Users/tina/Desktop/Drivers/chromedriver'
        driver = webdriver.Chrome(chrome_driver)
        driver.get('https://www.refinitiv.com/en/sustainable-finance/esg-scores')
        time.sleep(7)
              

df

<ipython-input-7-010387cf1c4e>:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


,Agilent Technologies Inc,Alcoa Corp,American Airlines Group Inc,Applied Optoelectronics Inc,AAON Inc,Advance Auto Parts Inc,Apple Inc,American Assets Trust Inc,Axon Enterprise Inc,AllianceBernstein Holding LP,...,Zendesk Inc,Olympic Steel Inc,Zoom Video Communications Inc,Zscaler Inc,ZTO Express (Cayman) Inc,Zoetis Inc,Zumiez Inc,Zuora Inc,Zynerba Pharmaceuticals Inc,Zynex Inc
Metric,,,,,,,,,,,,,,,,,,,,,
Environment,79,90,58,9,67,30,65,74,15,17,...,41,10,20,3,71,64,5,27,0,0
Emissions,96,94,66,0,74,60,96,85,32,42,...,85,7,44,14,73,64,12,57,0,0
Resource Use,95,94,75,23,47,11,98,76,12,35,...,65,15,30,0,92,83,0,45,0,0
Innovation,50,50,0,0,72,7,43,24,0,0,...,0,0,0,0,50,0,0,0,0,0
Social,91,84,66,21,42,72,78,47,36,68,...,76,46,66,55,60,78,29,60,35,34
Human Rights,92,72,45,0,0,70,79,55,13,73,...,82,33,35,0,48,68,44,73,0,0
Product Responsibiliy,91,64,39,42,37,61,54,41,43,45,...,35,94,35,35,52,64,25,35,30,73
Workforce,98,99,88,4,42,63,91,45,42,50,...,85,39,70,63,53,94,17,75,29,13
Community,84,94,74,48,96,91,89,52,55,93,...,98,61,99,87,84,87,33,67,50,35


In [10]:
df.index.rename('Company', inplace=True)
df

,Agilent Technologies Inc,Alcoa Corp,American Airlines Group Inc,Applied Optoelectronics Inc,AAON Inc,Advance Auto Parts Inc,Apple Inc,American Assets Trust Inc,Axon Enterprise Inc,AllianceBernstein Holding LP,...,Zendesk Inc,Olympic Steel Inc,Zoom Video Communications Inc,Zscaler Inc,ZTO Express (Cayman) Inc,Zoetis Inc,Zumiez Inc,Zuora Inc,Zynerba Pharmaceuticals Inc,Zynex Inc
Company,,,,,,,,,,,,,,,,,,,,,
Environment,79,90,58,9,67,30,65,74,15,17,...,41,10,20,3,71,64,5,27,0,0
Emissions,96,94,66,0,74,60,96,85,32,42,...,85,7,44,14,73,64,12,57,0,0
Resource Use,95,94,75,23,47,11,98,76,12,35,...,65,15,30,0,92,83,0,45,0,0
Innovation,50,50,0,0,72,7,43,24,0,0,...,0,0,0,0,50,0,0,0,0,0
Social,91,84,66,21,42,72,78,47,36,68,...,76,46,66,55,60,78,29,60,35,34
Human Rights,92,72,45,0,0,70,79,55,13,73,...,82,33,35,0,48,68,44,73,0,0
Product Responsibiliy,91,64,39,42,37,61,54,41,43,45,...,35,94,35,35,52,64,25,35,30,73
Workforce,98,99,88,4,42,63,91,45,42,50,...,85,39,70,63,53,94,17,75,29,13
Community,84,94,74,48,96,91,89,52,55,93,...,98,61,99,87,84,87,33,67,50,35


In [8]:
#save df to csv
df.to_csv('scraped_ESG_scores_final.csv')

In [18]:
df_trans = df.transpose()
df_trans.head()

,Environment,Emissions,Resource Use,Innovation,Social,Human Rights,Product Responsibiliy,Workforce,Community,Governance,Management,Shareholders,CSR Strategy,ESG Score
Company,,,,,,,,,,,,,,
Agilent Technologies Inc,79,96,95,50,91,92,91,98,84,90,88,97,87,89
Alcoa Corp,90,94,94,50,84,72,64,99,94,98,99,94,96,89
American Airlines Group Inc,58,66,75,0,66,45,39,88,74,85,78,99,95,69
Applied Optoelectronics Inc,9,0,23,0,21,0,42,4,48,12,14,11,0,14
AAON Inc,67,74,47,72,42,0,37,42,96,27,15,51,52,46


In [19]:
df_trans.to_csv('Refinitiv_ESG_scores_final.csv')

In [20]:
driver.quit()

In [24]:
#several Fortune 500 companies were not included during the scrape due to formatting issues, adding them manually

missing_cos = ["Amazon.com Inc","Chevron Corp","Home Depot Inc","Elevance Health Inc","Kroger Co","Meta Platforms Inc",
               "Lowe's Companies Inc","Procter & Gamble Co","Albertsons Companies Inc","Walt Disney Co",
               "Goldman Sachs Group Inc","Raytheon Technologies Corp","Boeing Co","Tesla Inc","Allstate Corp",
               "TJX Companies Inc","Progressive Corp","Stonex Group Inc","Plains GP Holdings LP","Thermo Fisher Scientific Inc",
               "Travelers Companies Inc","Qualcomm Inc","D.R. Horton Inc","Salesforce.Com Inc","Kraft Heinz Co","U.S. Bancorp",
               "Blackstone Group Inc","Hartford Financial Services Group Inc","Aflac Inc","ManpowerGroup Inc","PG&E Corp",
               "Carrier Global Corp","Baker Hughes Co","Sherwin-Williams Co","PNC Financial Services Group Inc","Kohls Corp",
               "Jones Lang LaSalle Inc","Charles Schwab Corp","Mastercard Inc","HF Sinclair Corp","Texas Instruments Inc",
               "Viatris Inc","H & R Block Inc","Goodyear Tire & Rubber Co","Targa Resources Corp","American Electric Power Company Inc",
               "Gap Inc","Estee Lauder Companies Inc","Laboratory Corporation of America Holdings","Otis Worldwide Corp",
               "Corning Inc","Ameriprise Financial Inc","O'Reilly Automotive Inc","Rocket Companies Inc","W W Grainger Inc",
               "Andersons Inc","Community Health Systems Inc","Mosaic Co","Dick's Sporting Goods Inc","Newmont Corporation",
               "Warner Bros Discovery Inc","J B Hunt Transport Services Inc","Vistra Corp","AES Corp","Caesars Entertainment Inc",
               "Equitable Holdings Inc","Quest Diagnostics Inc","Molson Coors Beverage Co","Interpublic Group of Companies Inc",
               "W. R. Berkley Corp","ODP Corp","Hershey Co","Dana Inc","Jackson Financial Inc","Carlyle Group Inc","Ovintiv Inc",
               "CommScope Holding Company Inc","Owens Corning","FM Global Logistics Holdings Bhd","Opendoor Technologies Inc",
               "J M Smucker Co","Anywhere Real Estate Inc","GXO Logistics Inc","APA Group","Boise Cascade Co","Bath & Body Works Inc",
               "T. Rowe Price Group Inc","Vertex Pharmaceuticals Inc","Arconic Corp (PITTSBURGH)","Science Applications International Corp",
               "Clorox Co","Travelcenters of America Inc","KLA Corp","Ingredion Inc","Academy Sports and Outdoors Inc","Dillard's Inc",
               "Compass Group PLC","Arko Corp.","Frontier Communications Parent Inc"]

In [21]:
df_missing = pd.DataFrame(index=['Environment', 'Emissions', 'Resource Use', 'Innovation', 'Social', 'Human Rights', 'Product Responsibiliy',
         'Workforce', 'Community', 'Governance', 'Management', 'Shareholders', 'CSR Strategy', 'ESG Score'])
df_missing.index.rename('Metric', inplace=True)    
print(df_missing)

Empty DataFrame
Columns: []
Index: [Environment, Emissions, Resource Use, Innovation, Social, Human Rights, Product Responsibiliy, Workforce, Community, Governance, Management, Shareholders, CSR Strategy, ESG Score]


In [26]:
# loop through the list of missing companies and append them to the new dataframe

chrome_driver = '/Users/tina/Desktop/Drivers/chromedriver'
driver = webdriver.Chrome(chrome_driver)

driver.get('https://www.refinitiv.com/en/sustainable-finance/esg-scores')
time.sleep(5)

testseries = []
myvals = []
counter = 0

for i in missing_cos:
    counter += 1
    element = driver.find_element(By.ID,"searchInput-1")
    element.send_keys(i)
    element.send_keys(Keys.RETURN)
    time.sleep(3)
    
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')
    
    myvals = soup.find_all("div", {"class": "table-align-right"})
    hidden = soup.find('div', hidden=True)
  
    time.sleep(1)
    
    if hidden == None:
        for x in myvals:
            testseries.append(x.string)
        myheaders = soup.find('h3', class_ = 'Heading Heading--m').find('strong').get_text()
        testseries.append(myheaders)
        df_missing[i] = testseries
    else:
        pass
    
    time.sleep(1)
    
    testseries = []
    
    ddelement = driver.find_element(By.CLASS_NAME, "SearchInput-clearButton")
    ddelement.click()
    
    time.sleep(2)
    
    if counter % 100 == 0:
        driver.quit()
        time.sleep(15)
        chrome_driver = '/Users/tina/Desktop/Drivers/chromedriver'
        driver = webdriver.Chrome(chrome_driver)
        driver.get('https://www.refinitiv.com/en/sustainable-finance/esg-scores')
        time.sleep(7)
              

df_missing

<ipython-input-26-29c4e3006cf5>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


,Amazon.com Inc,Chevron Corp,Home Depot Inc,Elevance Health Inc,Kroger Co,Meta Platforms Inc,Lowe's Companies Inc,Procter & Gamble Co,Albertsons Companies Inc,Walt Disney Co,...,Science Applications International Corp,Clorox Co,Travelcenters of America Inc,KLA Corp,Ingredion Inc,Academy Sports and Outdoors Inc,Dillard's Inc,Compass Group PLC,Arko Corp.,Frontier Communications Parent Inc
Metric,,,,,,,,,,,,,,,,,,,,,
Environment,89,81,86,85,85,49,73,85,44,57,...,38,55,1,50,46,0,21,74,6,9
Emissions,98,86,89,97,83,84,83,74,41,74,...,78,82,4,69,69,0,0,78,4,14
Resource Use,82,75,84,99,93,92,82,86,27,93,...,63,69,0,52,33,0,43,72,11,11
Innovation,80,81,86,0,78,0,44,99,78,0,...,0,0,0,33,0,0,13,67,0,0
Social,91,86,72,89,87,67,86,79,52,87,...,81,78,22,60,61,44,40,83,20,17
Human Rights,95,93,44,90,60,88,94,89,28,65,...,62,90,0,81,56,55,74,89,0,0
Product Responsibiliy,91,77,79,92,99,35,85,74,62,92,...,82,72,29,6,52,61,19,84,29,14
Workforce,88,77,62,93,87,96,68,84,35,87,...,86,66,35,68,62,12,36,63,28,23
Community,92,94,90,81,95,68,98,66,79,98,...,87,86,39,69,80,48,47,93,39,40


In [27]:
driver.quit()

In [32]:
df_missing_trans = df_missing.transpose()
df_missing_trans

Metric,Environment,Emissions,Resource Use,Innovation,Social,Human Rights,Product Responsibiliy,Workforce,Community,Governance,Management,Shareholders,CSR Strategy,ESG Score
Amazon.com Inc,89,98,82,80,91,95,91,88,92,81,84,86,60,87
Chevron Corp,81,86,75,81,86,93,77,77,94,97,98,96,94,87
Home Depot Inc,86,89,84,86,72,44,79,62,90,65,58,86,72,72
Elevance Health Inc,85,97,99,0,89,90,92,93,81,84,91,51,98,86
Kroger Co,85,83,93,78,87,60,99,87,95,65,65,60,72,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Academy Sports and Outdoors Inc,0,0,0,0,44,55,61,12,48,16,12,41,0,25
Dillard's Inc,21,0,43,13,40,74,19,36,47,24,20,56,0,29
Compass Group PLC,74,78,72,67,83,89,84,63,93,63,68,33,85,75
Arko Corp.,6,4,11,0,20,0,29,28,39,22,24,30,0,15


In [34]:
scraped_scores = pd.read_csv('scraped_ESG_scores_final.csv',index_col = [0])
scraped_trans = scraped_scores.transpose()
scraped_trans.head()

Metric,Environment,Emissions,Resource Use,Innovation,Social,Human Rights,Product Responsibiliy,Workforce,Community,Governance,Management,Shareholders,CSR Strategy,ESG Score
Agilent Technologies Inc,79.0,96.0,95.0,50.0,91.0,92.0,91.0,98.0,84.0,90.0,88.0,97.0,87.0,89.0
Alcoa Corp,90.0,94.0,94.0,50.0,84.0,72.0,64.0,99.0,94.0,98.0,99.0,94.0,96.0,89.0
American Airlines Group Inc,58.0,66.0,75.0,0.0,66.0,45.0,39.0,88.0,74.0,85.0,78.0,99.0,95.0,69.0
Applied Optoelectronics Inc,9.0,0.0,23.0,0.0,21.0,0.0,42.0,4.0,48.0,12.0,14.0,11.0,0.0,14.0
AAON Inc,67.0,74.0,47.0,72.0,42.0,0.0,37.0,42.0,96.0,27.0,15.0,51.0,52.0,46.0


In [35]:
#merge frames
df_final = pd.concat([df_missing_trans, scraped_trans], axis=0)
df_final

Metric,Environment,Emissions,Resource Use,Innovation,Social,Human Rights,Product Responsibiliy,Workforce,Community,Governance,Management,Shareholders,CSR Strategy,ESG Score
Amazon.com Inc,89,98,82,80,91,95,91,88,92,81,84,86,60,87
Chevron Corp,81,86,75,81,86,93,77,77,94,97,98,96,94,87
Home Depot Inc,86,89,84,86,72,44,79,62,90,65,58,86,72,72
Elevance Health Inc,85,97,99,0,89,90,92,93,81,84,91,51,98,86
Kroger Co,85,83,93,78,87,60,99,87,95,65,65,60,72,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoetis Inc,64.0,64.0,83.0,0.0,78.0,68.0,64.0,94.0,87.0,56.0,55.0,60.0,55.0,68.0
Zumiez Inc,5.0,12.0,0.0,0.0,29.0,44.0,25.0,17.0,33.0,30.0,39.0,19.0,0.0,25.0
Zuora Inc,27.0,57.0,45.0,0.0,60.0,73.0,35.0,75.0,67.0,48.0,59.0,17.0,40.0,50.0
Zynerba Pharmaceuticals Inc,0.0,0.0,0.0,0.0,35.0,0.0,30.0,29.0,50.0,19.0,17.0,39.0,0.0,20.0


In [42]:
df_final.to_csv('Refinitiv_final_ESG_scores_Oct_15_2022.csv')